##### DNN for Hand Recognition

TODO: 
1. Optimize hyperparams via (https://www.tensorflow.org/tutorials/keras/keras_tuner) 
2. Filter outliers in training data 
3. Data Augmentation
4. NN Pruning
5. Speed up data generation

# Imports and Training Params

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
from multiprocessing import Pool
import matplotlib.pyplot as plt
import json
import zipfile
import os
import gc
import warnings
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

BASE_TRAIN_PATH = "/kaggle/input/asl-signs/"
TRAIN_FILE = "/kaggle/input/asl-signs/train.csv"
INDEX_MAP_FILE = '/kaggle/input/asl-signs/sign_to_prediction_index_map.json'
MODEL_OUT_PATH = '/kaggle/working/modelie'

data_columns = ['x', 'y', 'z']

use_generator = False 
find_optimal_params = True
filter_outliers = True 
training = True
local_inference_test = False

epochs = 50
batch_size = 100

rows_per_frame = 543 #Number of landmarks per frame 
num_classes = 250 
dropout_rate = .3

max_length = 30 # length that input is padded/truncated to 

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

LEFT_HAND_OFFSET = 468
POSE_OFFSET = LEFT_HAND_OFFSET+21
RIGHT_HAND_OFFSET = POSE_OFFSET+33
ROWS_PER_FRAME = 543
lip_landmarks = [61, 185, 40, 39, 37,  0, 267, 269, 270, 409,
                 291,146, 91,181, 84, 17, 314, 405, 321, 375, 
                 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 
                 95, 88, 178, 87, 14,317, 402, 318, 324, 308]

left_hand_landmarks = list(range(LEFT_HAND_OFFSET, LEFT_HAND_OFFSET+21))
right_hand_landmarks = list(range(RIGHT_HAND_OFFSET, RIGHT_HAND_OFFSET+21))

point_landmarks = [item for sublist in [lip_landmarks, left_hand_landmarks, right_hand_landmarks] for item in sublist]

train = pd.read_csv(TRAIN_FILE)
with open(INDEX_MAP_FILE, 'r') as f: 
    index_map = json.load(f)

train['label'] = train['sign'].map(lambda x: index_map[x])

# Multiprocessed Data Loading (to Disk)

In [ ]:
if not use_generator: 
    train_paths = [train.iloc[i].path for i in range(len(train))]
    y = [train.iloc[i].label for i in range(len(train))]

    def fill_x(path): 
        partial_x = []
        data = pd.read_parquet(os.path.join(BASE_TRAIN_PATH, path), columns=data_columns)

        data.fillna(0, inplace=True)

        n = int(len(data)/rows_per_frame)

        data = data.values.reshape(n, rows_per_frame, len(data_columns)).astype(np.float32)
        data = data[:, point_landmarks, :] # filter for the relevant pose landmarks 
        
        data = np.nan_to_num(data) # Should not have any Nans here 
        data = np.reshape(data, (n, len(point_landmarks)*3), order = 'C').astype(np.float32)
        return data

    with Pool(processes=10) as pool: 
        x_ragged = list(tqdm(pool.imap(fill_x, train_paths, chunksize=1000), total = 100))
        pool.close()

    x_ragged = np.array(x_ragged)
    y = np.array(y)

    x = tf.keras.utils.pad_sequences(x_ragged, padding="post", truncating="post", maxlen = max_length, dtype=np.float32)

    del x_ragged 
    gc.collect()
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) 

    del x , y
    gc.collect()

# Data Generator
Allows us to load more data than can fit in memory

In [ ]:
class DataGenerator(tf.keras.utils.Sequence): 
    def __init__(self, train, list_IDs, point_landmarks, batch_size=32, max_length=30, rows_per_frame=543, 
                 data_columns=['x','y','z'], shuffle=True):
        self.train = train
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.max_length = max_length 
        self.rows_per_frame = rows_per_frame
        self.point_landmarks = point_landmarks
        self.data_columns = data_columns 
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, list_IDs_temp): 
        X = np.empty((self.batch_size, self.max_length, len(self.point_landmarks)*len(self.data_columns)))
        y = np.empty((self.batch_size), dtype=int)
        
        for i, ID in enumerate(list_IDs_temp):
            data = pd.read_parquet(os.path.join(BASE_TRAIN_PATH, self.train.iloc[ID].path), columns=self.data_columns)
          
            data.fillna(0, inplace=True)
            n = int(len(data)/self.rows_per_frame)
            data = data.values.reshape(n, self.rows_per_frame, len(self.data_columns)).astype(np.float32)
            data = data[:, self.point_landmarks, :]
            data = np.reshape(data, (n, len(self.point_landmarks)*len(self.data_columns)), order = 'C').astype(np.float32)
            
            if data.shape[0] > self.max_length:  
                data = data[:self.max_length, :]
            elif data.shape[0] < self.max_length: 
                data = np.pad(data, ((0,self.max_length - data.shape[0]),(0,0)))
                
            X[i,] = data
            y[i] = self.train.iloc[ID].label
            
        return X, y
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)
        return X, y

if use_generator:     
    ds_params = {
        'max_length': max_length, 
        'batch_size': batch_size, 
        'rows_per_frame': rows_per_frame,
        'data_columns': data_columns, 
        'shuffle': True}

    partition = {'train': [i for i in range(int(len(train)*.8))], 'validation': [j for j in range(int(len(train)*.8), len(train))]}

    training_generator = DataGenerator(train, partition['train'], point_landmarks, **ds_params)
    validation_generator = DataGenerator(train, partition['validation'], point_landmarks, **ds_params)

### Frames Stats

Mean: 37.935

Median: 22

StdDev: 44.177

Max: 537

Min: 2

In [ ]:
def get_model(): 
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(max_length, len(point_landmarks)*3), dtype=np.float32), 
        tf.keras.layers.Masking(mask_value=0, input_shape=(max_length, len(point_landmarks)*3)),
        tf.keras.layers.Dense(units=256, activation='relu'), # Generally want hidden layers to be between the size of the input and output layers 
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LayerNormalization(), 
        tf.keras.layers.Dense(units=256, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LSTM(256),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = num_classes, activation='softmax', name='outie'), # Output size is <256>
    ])
    model.compile(optimizer = "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model 

model = get_model() 

# Optimal Hyper Parameter Search

In [ ]:
#Hyper Param tuning 
!pip install -q -U keras-tuner
import keras_tuner as kt 

def get_tuned_model(hp): 
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
    hp_units_3 = hp.Int('units_3', min_value=32, max_value=512, step=32)
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(max_length, len(point_landmarks)*3), dtype=np.float32), 
        tf.keras.layers.Masking(mask_value=0, input_shape=(max_length, len(point_landmarks)*3)),
        tf.keras.layers.Dense(units=hp_units, activation='relu'), # Generally want hidden layers to be between the size of the input and output layers 
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LayerNormalization(), 
        tf.keras.layers.Dense(units=hp_units_2, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LSTM(hp_units_3),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = num_classes, activation='softmax', name='outie'), # Output size is <256>
    ])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model 

if find_optimial_params: 
    tuner = kt.Hyperband(get_tuned_model,
                         objective='val_accuracy',
                         max_epochs=10,
                         factor=3,
                         directory='kaggle/working',
                         project_name='intro_to_kt')

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    tuner.search(X_train, Y_train, validation_data=(X_val, Y_val), epochs=50, callbacks=[stop_early])

    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    print(best_hps.get('units'))
    print(best_hps.get('units2'))
    print(best_hps.get('units3'))
    print(best_hps.get('learning_rate'))
    model = tuner.hypermodel.build(best_hps)

    history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=50)
    val_acc_per_epoch = history.history['val_accuracy']
    best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
    print('Best epoch: %d' % (best_epoch,))
    epochs = best_epoch

# Model fitting using either Data Generator or Numpy Arrays

In [ ]:
callbacks = [
        EarlyStopping(
                monitor = "val_accuracy",
                min_delta = 0, # minimium amount of change to count as an improvement
                patience = 5, # how many epochs to wait before stopping
                restore_best_weights=True),
    
        ReduceLROnPlateau(monitor = "val_accuracy",
            factor = 0.5,
            patience = 5)
            ]

if use_generator: 
    history = model.fit(x=training_generator, 
                        validation_data=validation_generator, 
                        epochs=epochs,
                        callbacks=callbacks, 
                        use_multiprocessing=True, 
                        workers=6)

    loss, acc = model.evaluate(validation_generator, verbose=2)
else: 
    history = model.fit(X_train, Y_train, 
                  batch_size=batch_size, 
                  epochs=epochs,
                  validation_data = (X_val, Y_val), 
                  callbacks=callbacks)

    loss, acc = model.evaluate(X_val, Y_val, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# Model Summary
## Loss and Accuracy Graphs

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Inference Model

In [ ]:
class FeatureGenTF(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        x = tf.gather(x, point_landmarks, axis=1)
        x = tf.image.resize_with_pad(x, max_length, len(point_landmarks))
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x = tf.reshape(x, (max_length, len(point_landmarks)*3))
        x = tf.expand_dims(x,0) # THIS IS SAYING BATCH SIZE ONE TO MODEL YEAAAA

        return x
    
preprocessing = FeatureGenTF()

In [ ]:
def get_inference_model(model):
    inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs") 
    x = preprocessing(inputs)
    x = model(x)
    output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output) 
    inference_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    return inference_model

inference_model = get_inference_model(model)
inference_model.summary(expand_nested=True)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
    f.write(tflite_model)

print(f"Model Size: {os.stat('model.tflite').st_size}")

!zip submission.zip $model_path

# Testing inference model

In [ ]:
!pip install tflite_runtime
import tflite_runtime.interpreter as tflite
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

if local_inference_test: 
    #TODO Find out what signs the model is bad at inferring
    interpreter = tflite.Interpreter(model_path)
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")
    p2s_map = {v: k for k, v in index_map.items()}
    decoder = lambda x: p2s_map.get(x)
    score = 0 
    for i in range(int(len(train)/10)): 
        frames = load_relevant_data_subset(os.path.join(BASE_TRAIN_PATH, train.iloc[i].path))
        output = prediction_fn(inputs=frames)
        sign = np.argmax(output["outputs"])
        if decoder(train.iloc[i].label) == decoder(sign): 
            score += 1 
    print((len(train)/10)/score)